<center><h1>Job Recommendation</h1>

<center><h1>Naukri Dataset</h1>

##### Source: https://www.kaggle.com/PromptCloudHQ/jobs-on-naukricom

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install patool
!pip install pyunpack

     |████████████████████████████████| 81kB 2.6MB/s 


In [ ]:
from pyunpack import Archive

Archive('/content/drive/My Drive/Naukri Jobs Dataset/jobs-on-naukricom.zip').extractall('/content')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import copy

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
naukri = pd.read_csv('/content/naukri_com-job_sample.csv')

In [ ]:
print("Shape of naukri dataset:", naukri.shape)

print("Column names of naukri dataset:\n")
naukri.columns

Shape of naukri dataset: (22000, 14)
Column names of naukri dataset:



Index(['company', 'education', 'experience', 'industry', 'jobdescription',
       'jobid', 'joblocation_address', 'jobtitle', 'numberofpositions',
       'payrate', 'postdate', 'site_name', 'skills', 'uniq_id'],
      dtype='object')

In [ ]:
print("Head of dataframe.\n")

naukri.head()

Head of dataframe.



,company,education,experience,industry,jobdescription,jobid,joblocation_address,jobtitle,numberofpositions,payrate,postdate,site_name,skills,uniq_id
0,MM Media Pvt Ltd,UG: B.Tech/B.E. - Any Specialization PG:Any Po...,0 - 1 yrs,Media / Entertainment / Internet,Job Description Send me Jobs like this Quali...,210516002263,Chennai,Walkin Data Entry Operator (night Shift),NaN,"1,50,000 - 2,25,000 P.A",2016-05-21 19:30:00 +0000,NaN,ITES,43b19632647068535437c774b6ca6cf8
1,find live infotech,UG: B.Tech/B.E. - Any Specialization PG:MBA/PG...,0 - 0 yrs,Advertising / PR / MR / Event Management,Job Description Send me Jobs like this Quali...,210516002391,Chennai,Work Based Onhome Based Part Time.,60.0,"1,50,000 - 2,50,000 P.A. 20000",2016-05-21 19:30:00 +0000,NaN,Marketing,d4c72325e57f89f364812b5ed5a795f0
2,Softtech Career Infosystem Pvt. Ltd,UG: Any Graduate - Any Specialization PG:Any P...,4 - 8 yrs,IT-Software / Software Services,Job Description Send me Jobs like this - as ...,101016900534,Bengaluru,Pl/sql Developer - SQL,NaN,Not Disclosed by Recruiter,2016-10-13 16:20:55 +0000,NaN,IT Software - Application Programming,c47df6f4cfdf5b46f1fd713ba61b9eba
3,Onboard HRServices LLP,UG: Any Graduate - Any Specialization PG:CA Do...,11 - 15 yrs,Banking / Financial Services / Broking,Job Description Send me Jobs like this - Inv...,81016900536,"Mumbai, Bengaluru, Kolkata, Chennai, Coimbator...",Manager/ad/partner - Indirect Tax - CA,NaN,Not Disclosed by Recruiter,2016-10-13 16:20:55 +0000,NaN,Accounts,115d28f140f694dd1cc61c53d03c66ae
4,Spire Technologies and Solutions Pvt. Ltd.,UG: B.Tech/B.E. - Any Specialization PG:Any Po...,6 - 8 yrs,IT-Software / Software Services,Job Description Send me Jobs like this Pleas...,120916002122,Bengaluru,JAVA Technical Lead (6-8 yrs) -,4.0,Not Disclosed by Recruiter,2016-10-13 16:20:55 +0000,NaN,IT Software - Application Programming,a12553fc03bc7bcced8b1bb8963f97b4


## Check for null values

In [ ]:
naukri.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   company              21996 non-null  object 
 1   education            20004 non-null  object 
 2   experience           21996 non-null  object 
 3   industry             21995 non-null  object 
 4   jobdescription       21996 non-null  object 
 5   jobid                22000 non-null  int64  
 6   joblocation_address  21499 non-null  object 
 7   jobtitle             22000 non-null  object 
 8   numberofpositions    4464 non-null   float64
 9   payrate              21903 non-null  object 
 10  postdate             21977 non-null  object 
 11  site_name            3987 non-null   object 
 12  skills               21472 non-null  object 
 13  uniq_id              22000 non-null  object 
dtypes: float64(1), int64(1), object(12)
memory usage: 2.3+ MB


## Observation

- We don't want to lose jobs so we will not remove null values.

### Plan of action for null values.
- We will fill the null values with N/A except skills and description columns.
- For description columns, we will fill null values with the corresponding job title to make it more meaningful.
- We will not fill null values of skills with N/A. We will train a model with target variable as skills and predict the skills that is not given.

In [ ]:
import copy
naukri_c = copy.deepcopy(naukri)

In [ ]:
# dummy dataframe with jobdescription and skills
des_skill = naukri_c[['jobdescription', 'skills']]

# dropping job description and skills columns
naukri_c.drop(['jobdescription', 'skills'], axis=1, inplace=True)

In [ ]:
# Filling null values with N/A

naukri_c.fillna('N/A', inplace=True)

naukri_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   company              22000 non-null  object
 1   education            22000 non-null  object
 2   experience           22000 non-null  object
 3   industry             22000 non-null  object
 4   jobid                22000 non-null  int64 
 5   joblocation_address  22000 non-null  object
 6   jobtitle             22000 non-null  object
 7   numberofpositions    22000 non-null  object
 8   payrate              22000 non-null  object
 9   postdate             22000 non-null  object
 10  site_name            22000 non-null  object
 11  uniq_id              22000 non-null  object
dtypes: int64(1), object(11)
memory usage: 2.0+ MB


- All the null values have been filled with N/A.
- Create new columns in naukri_c (jobdescription and skills) that will have values from des_skills.
- Then fill null values of jobdescription with corresponding titles.
- Note: Keep the null values of skills as is. We will train model with given skills and predict the null values of skills.

In [ ]:
# Create new columns in naukri_c (jobdescription and skills) that will have values from des_skills.
naukri_c['jobdescription'] = des_skill['jobdescription']
naukri_c['skills'] = des_skill['skills']

In [ ]:
# we will get the list of index values that have null values for job description
list_index_naukri_c = list(des_skill[des_skill['jobdescription'].isnull()==True].index)

print("Index values that have null vaues for description:")
list_index_naukri_c

Index values that have null vaues for description:


[3768, 4026, 4389, 4841]

In [ ]:
# Now we will fill job description possessing the indexes from the list_index_naukri_c with the corresponding title.

for idx in list_index_naukri_c:
    naukri_c['jobdescription'][idx] = naukri_c['jobtitle'][idx]

In [ ]:
# cross verifying if there are any null values in the description
naukri_c[naukri_c['jobdescription'].isnull()==True]

,company,education,experience,industry,jobid,joblocation_address,jobtitle,numberofpositions,payrate,postdate,site_name,uniq_id,jobdescription,skills


### No null values under job description column. It has been filled with corresponding job title.

## Checking for duplicates jobid and job title

In [ ]:
# check for duplicates under jobid

naukri_c[naukri_c['jobid'].duplicated()==True]

,company,education,experience,industry,jobid,joblocation_address,jobtitle,numberofpositions,payrate,postdate,site_name,uniq_id,jobdescription,skills
5154,ScaleneWorks People Solutions LLP,UG: B.Tech/B.E. - Any Specialization PG:M.Tech...,3 - 6 yrs,Telecom/ISP /,231115004567,"Bengaluru/Bangalore , Chennai",Opening for UI Developer - Bangalore/ Chennai,5,"2,75,000 - 7,75,000 P.A",2015-11-23 01:33:39 +0000,N/A,e772dedfb5ea5c91bc1292557b793457,Job Description Send me Jobs like this Dear ...,IT Software - Application Programming
6771,Diksha Technologies,UG: Any Graduate PG:Any Postgraduate Doctorate...,8 - 13 yrs,IT-Software / Software Services,280515504569,"Bengaluru/Bangalore , Gurgaon",Intec Interconnect /IOA v7 Experts,N/A,Not Disclosed by Recruiter,2016-01-29 14:07:53 +0000,N/A,49b41ee4b87d8c85b0bc7154308bc237,Job Description Send me Jobs like this Stron...,IT Software - Application Programming
10215,Adecco India,UG: Any Graduate - Any Specialization PG:Any P...,5 - 10 yrs,IT-Software / Software Services,290316900910,Bengaluru/Bangalore,Sr. Technical Lead with a Product Based Company,N/A,Not Disclosed by Recruiter,2016-03-29 17:04:12 +0000,N/A,232c5cb46d3600d905ff74f6b268086b,Job Description Send me Jobs like this Looki...,IT Software - Application Programming
10541,White Horse Manpower Consultancy Private Limited,"UG: Any Graduate - Any Specialization, Graduat...",0 - 4 yrs,BPO / Call Centre / ITES,30315002273,Bengaluru/Bangalore,Spot Offer for BPO Day/uk Shift Immediate Join...,N/A,"1,25,000 - 4,00,000 P.A. Plus Incentives and f...",2016-03-30 17:04:32 +0000,N/A,3487f4defbb93b1c0cd098d93b152ff9,Job Description Send me Jobs like this * Can...,ITES
12511,RATEIN INFOTECH (I) PVT. LTD.,UG: B.Tech/B.E. PG:M.Tech,7 - 9 yrs,IT-Software / Software Services,51116500258,"Gurgaon, Mumbai",Oracle,N/A,Not Disclosed by Recruiter,2016-11-05 19:48:40 +0000,www.naukri.com,246fac8666a37a6bad4dfc8add92b71b,Job Description Send me Jobs like this Job D...,IT Software - Application Programming
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21476,Anantha Cyber Tech Pvt Ltd,N/A,4 - 6 yrs,IT-Software / Software Services,221216001952,Hyderabad,Hiring for Application Packaging Consultant at...,N/A,Not Disclosed by Recruiter,2016-12-22 18:19:34 +0000,www.naukri.com,2aae653270573288d25fce36c869fd80,Job Description Send me Jobs like this Hi Al...,IT Software - Application Programming
21546,K SQUARE INDIA TECHNOLOGIES PVT LTD,N/A,5 - 9 yrs,IT-Software / Software Services,221216001982,"Mumbai, Bengaluru, Hyderabad",Need SAP FICO / MM / SD with CIN,N/A,Not Disclosed by Recruiter,2016-12-22 18:19:37 +0000,www.naukri.com,e430b9c5cc61f03222703e7505505f0b,Job Description Send me Jobs like this We re...,IT Software - Application Programming
21589,Cucumber Consultants,PG:Any Postgraduate,5 - 8 yrs,Retail / Wholesale,121116001131,Hyderabad,Operations Manager-hyderabad,N/A,"4,00,000 - 6,00,000 P.A",2016-12-22 18:19:39 +0000,www.naukri.com,91214d68455a6a77ceacdf61d68f47a5,Job Description Send me Jobs like this Candi...,ITES
21634,Fortune HR Consultants hiring for Leading Gene...,UG: Any Graduate - Any Specialization,2 - 7 yrs,Banking / Financial Services / Broking,311015002604,"Bengaluru, Chennai, Hyderabad",Unit Sales Manager - Key Relations Group - Ban...,N/A,"3,00,000 - 5,50,000 P.A. Reimbursement and Bonus",2016-12-21 18:19:41 +0000,www.naukri.com,936a7402e8726cde46021f8e653ecea5,Job Description Send me Jobs like this Drivi...,Sales


### Let us check duplicates size for job id, job title, job description and subset of job id & title

In [ ]:
# check for duplicates under jobid
print("Number of duplicates in job id:", naukri_c[naukri_c['jobid'].duplicated()==True].shape[0])

# check for duplicates under job title
print("Number of duplicates in job title:", naukri_c[naukri_c['jobtitle'].duplicated()==True].shape[0])

# check for duplicates under job description
print("Number of duplicates in job description:", naukri_c[naukri_c['jobdescription'].duplicated()==True].shape[0])

subset_id_title = naukri_c[(naukri_c['jobid'].duplicated()==True) & (naukri_c['jobtitle'].duplicated()==True)]
print("Number of duplicates for subset of id and title:", subset_duplicate.shape[0])

Number of duplicates in job id: 90
Number of duplicates in job title: 4436
Number of duplicates in job description: 933
Number of duplicates for subset of id and title: 88


## Observation
- 90 duplicates in job id.
- 4436 duplicates in job title
- 936 duplicates in job description
- 88 duplicates in subset of id and title.

### Plan of action
- First, we will sort the dataframe by job ids.
- Second, we will remove duplicates of id.

### Note: We will not remove duplicates of title and description as we will lose job id data. 

In [ ]:
# sorting the dataframe by job ids
naukri_sort = naukri_c.sort_values('jobid', axis=0, ascending=True)

# removing duplicates that are subset of subset of id.
naukri_dup = naukri_sort.drop_duplicates(subset={'jobid'}, keep='first')

print("Data percentage reduced to after removing duplicates:", round(100*(naukri_dup.shape[0]/naukri_c.shape[0]), 4))
print("Shape of dataframe after removing duplicates:", naukri_dup.shape)


Data percentage reduced to after removing duplicates: 99.5909
Shape of dataframe after removing duplicates: (21910, 14)


In [ ]:
# check for duplicates under jobid
print("Number of duplicates in job id:", naukri_dup[naukri_dup['jobid'].duplicated()==True].shape[0])

Number of duplicates in job id: 0


In [ ]:
## As we have removed duplcates, index will be unorder. We will reset index
naukri_dup.reset_index(inplace=True, drop=True)

<center><h1>Skills prediction for the null values</h1>

In [ ]:
naukri.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   company              21996 non-null  object 
 1   education            20004 non-null  object 
 2   experience           21996 non-null  object 
 3   industry             21995 non-null  object 
 4   jobdescription       21996 non-null  object 
 5   jobid                22000 non-null  int64  
 6   joblocation_address  21499 non-null  object 
 7   jobtitle             22000 non-null  object 
 8   numberofpositions    4464 non-null   float64
 9   payrate              21903 non-null  object 
 10  postdate             21977 non-null  object 
 11  site_name            3987 non-null   object 
 12  skills               21472 non-null  object 
 13  uniq_id              22000 non-null  object 
dtypes: float64(1), int64(1), object(12)
memory usage: 2.3+ MB


## Columns selection
- We don't need all the columns. We will select few columns that we think has the largest impact on the target variable (skills).
- As only jobid is of type int and rest of the columns are of object type, we can't use heatmap visualization to see the correlation of columns.
- So we will with our domain knowledge, select the columns that we think has the largest impact on the target variable (skills).
- Considering only jobid, jobtitle and jobdescription columns.

In [ ]:
# naukri_dup.to_csv('/content/drive/My Drive/Naukri Jobs Dataset/naukri_no_duplicate.csv', index=False)



In [ ]:
train = naukri_dup[['jobid', 'jobtitle', 'jobdescription', 'skills']]

print("Shape of train:", train.shape)

print("\n\nHead of train\n")
train.head()

Shape of train: (21910, 4)


Head of train



,jobid,jobtitle,jobdescription,skills
0,10116001126,Recruitment Coordinator- Bangalore,Job Description Send me Jobs like this Job D...,HR
1,10116001286,Medical Coding- Fresher to CPC Coder- Whatsapp...,Job Description Send me Jobs like this YOUR ...,ITES
2,10216001911,Biz Development & Delivery- Infrastructure Pra...,Job Description Send me Jobs like this Respo...,HR
3,10216001918,Sales Executive,Job Description Send me Jobs like this Good...,Sales
4,10216005682,"Fresher CA Job-big 4 Shared Services, Gurgaon-...",Job Description Send me Jobs like this Emplo...,Accounts


In [ ]:
train.tail()

,jobid,jobtitle,jobdescription,skills
21905,311215002194,Mixed Signal Verification Engineer - Bangalore...,Job Description Send me Jobs like this Hi G...,IT Software - Application Programming
21906,311215002974,Tivoli Monitoring -ITM,Job Description Send me Jobs like this We ha...,IT Software - Client/Server Programming
21907,311215003695,Assistant Manager/manager - Safety,Job Description Send me Jobs like this Job O...,NaN
21908,311215004291,Hiring for Value Engineering Professionals in ...,Job Description Send me Jobs like this Shift...,NaN
21909,311215500050,Chip Lead,Job Description Send me Jobs like this Respo...,IT Software - Embedded


## Observation:
- We can see that the sentence 'Job Description  Send me jobs like this' is for all the jobs.
- We will remove the above mentioned sentence.

In [ ]:
train['jobdescription'] = train['jobdescription'].apply(lambda x: x.split('this')[1] if len(x.split('this'))>1 else x.split('this')[0])

In [ ]:
train.head()

,jobid,jobtitle,jobdescription,skills
0,10116001126,Recruitment Coordinator- Bangalore,Job Description Heres what you will do as par...,HR
1,10116001286,Medical Coding- Fresher to CPC Coder- Whatsapp...,YOUR CAREER CLAPPING CALLS ENDS HERE @ YOGAM ...,ITES
2,10216001911,Biz Development & Delivery- Infrastructure Pra...,Responsible for market research and market ex...,HR
3,10216001918,Sales Executive,"Good communication, Graduate with 2 year of ...",Sales
4,10216005682,"Fresher CA Job-big 4 Shared Services, Gurgaon-...",Employer : Big 4 ( Shared Services) Position ...,Accounts


## Data cleaning
- Removing punctuations.
- Changing the strings to lowercase.
- Getting the root word with porter stemmer or lemmatization.

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
from bs4 import BeautifulSoup

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
s = SnowballStemmer('english')
lem = WordNetLemmatizer()

In [ ]:
def clean(text):

    te = BeautifulSoup(text, 'lxml')
    te = re.sub('[^a-zA-Z]', ' ', text)
    # te = ' '.join(lem.lemmatize(sw) for sw in te.split() if sw not in stopwords.words('english'))
    te = ' '.join(sw for sw in te.split() if sw not in stopwords.words('english'))
    te = te.lower().strip()

    return te

### Uncomment and run the below 2 cells

In [3]:
# Uncomment and run the cell

# from tqdm import tqdm
# clean_title = []

# for i, s in enumerate (tqdm(train['jobtitle'])):
#     t = clean(s)
#     clean_title.append(t)

In [4]:
# Uncomment and run the cell

# clean_des = []

# for i, s in enumerate (tqdm(train['jobdescription'])):
#     t = clean(s)
#     clean_des.append(t)

In [ ]:
train_clean = copy.deepcopy(train)

In [ ]:
print("Assigning clean text to title and description columns...\n")

train_clean['jobtitle']= clean_title
train_clean['jobdescription'] = clean_des

train_clean.head()

Assigning clean text to title and description columns...



,jobid,jobtitle,jobdescription,skills
0,10116001126,recruitment coordinator bangalore,job description heres part recruitment team as...,HR
1,10116001286,medical coding fresher cpc coder whatsapp yoga...,your career clapping calls ends here yogam bpo...,ITES
2,10216001911,biz development delivery infrastructure practi...,responsible market research market expansion i...,HR
3,10216001918,sales executive,good communication graduate year experience te...,Sales
4,10216005682,fresher ca job big shared services gurgaon int...,employer big shared services position senior a...,Accounts


In [ ]:
# train_clean.to_csv('/content/drive/My Drive/Naukri Jobs Dataset/naukri_train_clean.csv', index=False)

train_clean = pd.read_csv('/content/drive/My Drive/Jobs_Recommend_Project/Copy of naukri_train_clean.csv')

In [ ]:
test_clean = train_clean[train_clean['skills'].isnull()==True]

print("Shape of test_clean:", test_clean.shape)

Shape of test_clean: (527, 4)


In [ ]:
test_index_list = list(test_clean.index)
test_index_list[:10]

[224, 304, 311, 312, 320, 321, 322, 380, 422, 456]

In [ ]:
test_clean['jobdescription'].fillna('job description not mentioned', inplace=True)

## dropping null data based on test indexes

In [ ]:
print("Shape of train_clean before dropping na:", train_clean.shape)

train_clean.drop(index=test_index_list, inplace=True)

print("Shape of train_clean after dropping na:", train_clean.shape)

Shape of train_clean before dropping na: (21910, 4)
Shape of train_clean after dropping na: (21383, 4)


## Feature engineering
- combining title and description

In [ ]:
train_clean['ti_de'] = train_clean['jobtitle'] + ' ' + train_clean['jobdescription']
test_clean['ti_de'] = test_clean['jobtitle'] + ' ' + test_clean['jobdescription']

## Encoding target variable
- With keys as skills and values as int,

In [ ]:
print("Number of unique skills:", train_clean['skills'].nunique())

Number of unique skills: 45


In [ ]:
train_clean.skills.value_counts()

IT Software - Application Programming      5969
Sales                                      2871
ITES                                       1622
Teaching                                   1091
HR                                          927
Marketing                                   865
Accounts                                    855
Production                                  662
Medical                                     416
Financial Services                          410
IT Software - Other                         407
Engineering Design                          381
IT Software - Network Administration        372
IT Software - ERP                           353
IT Software - QA & Testing                  341
IT Software - eCommerce                     335
Site Engineering                            322
IT Software - DBA                           273
IT Software - Embedded                      261
IT Hardware                                 253
Journalism                              

In [ ]:
print("Storing unique skills in a list to create a dictionary for mapping to skills to encode.\n")

skills_list_uni = list(train_clean['skills'].unique())

print("Length of unique skills list:", len(skills_list_uni), '\n')
print("First 10 unique skills:", skills_list_uni[:10])


Storing unique skills in a list to create a dictionary for mapping to skills to encode.

Length of unique skills list: 45 

First 10 unique skills: ['HR', 'ITES', 'Sales', 'Accounts', 'IT Software - ERP', 'IT Software - Application Programming', 'Design', 'Marketing', 'Production', 'Journalism']


In [ ]:
skills_key = {}
skills_val = {}
for i in range(len(skills_list_uni)):

    # skills as key and int as values
    skills_key[skills_list_uni[i]] = i

    # int as keys and skills as values
    skills_val[i] = skills_list_uni[i]

In [ ]:
print("Shape of skills_key:", len(skills_key), '\n')
print("First 5 items of skills_key:")
print(dict(list(skills_key.items())[:5]))
                                         
print("\n\nShape of skills_val:", len(skills_val), '\n')
print("First 5 items of skills_val:")
print(dict(list(skills_val.items())[:5]))
                                         

Shape of skills_key: 45 

First 5 items of skills_key:
{'HR': 0, 'ITES': 1, 'Sales': 2, 'Accounts': 3, 'IT Software - ERP': 4}


Shape of skills_val: 45 

First 5 items of skills_val:
{0: 'HR', 1: 'ITES', 2: 'Sales', 3: 'Accounts', 4: 'IT Software - ERP'}


In [ ]:
# train_clean_c = copy.deepcopy(train_clean)

## Creating a new column 'skills_encoded' that will have encoded skills

In [ ]:
# mapping skills_encoded with skills_dict to encode
train_clean['skills'] = train_clean['skills'].map(skills_key)

# head of dataframe
train_clean.head()

,jobid,jobtitle,jobdescription,skills,ti_de
0,10116001126,recruitment coordinator bangalore,job description heres part recruitment team as...,0,recruitment coordinator bangalore job descript...
1,10116001286,medical coding fresher cpc coder whatsapp yoga...,your career clapping calls ends here yogam bpo...,1,medical coding fresher cpc coder whatsapp yoga...
2,10216001911,biz development delivery infrastructure practi...,responsible market research market expansion i...,0,biz development delivery infrastructure practi...
3,10216001918,sales executive,good communication graduate year experience te...,2,sales executive good communication graduate ye...
4,10216005682,fresher ca job big shared services gurgaon int...,employer big shared services position senior a...,3,fresher ca job big shared services gurgaon int...


In [ ]:
# train_clean.fillna('N/A', inplace = True)

In [ ]:
# train data
x_train = train_clean[['jobtitle', 'jobdescription', 'ti_de']]
y_train = train_clean['skills']

# test data
x_test = test_clean[['jobtitle', 'jobdescription', 'ti_de']]
y_test = test_clean['skills']

print("Shape of x_train:", x_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of x_test:", x_test.shape)
print("Shape of y_test:", y_test.shape)

Shape of x_train: (21383, 3)
Shape of y_train: (21383,)
Shape of x_test: (527, 3)
Shape of y_test: (527,)


## Multiclass classification
- Since it is multiclass classification, we have to use onevsrest classifier with logistic regression algorithm or any other algorithm.

## vectorization title

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_t = TfidfVectorizer(min_df=2)

x_tr_t = tf_t.fit_transform(x_train['jobtitle'])
x_te_t = tf_t.transform(x_test['jobtitle'])

vectorization description

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_d = TfidfVectorizer(min_df=2, ngram_range=(1,2))

x_tr_d = tf_d.fit_transform(x_train['jobdescription'])
x_te_d = tf_d.transform(x_test['jobdescription'])

vectorization ti_de

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_td = TfidfVectorizer(min_df=2, ngram_range=(1,2))

x_tr_td = tf_td.fit_transform(x_train['ti_de'])
x_te_td = tf_td.transform(x_test['ti_de'])

## saving vectorized data in pickle file for future use


In [ ]:
import pickle

# saving vectorized title
pickle.dump(tf_t, open('/content/drive/My Drive/Jobs_Recommend_Project/naukri_tf_title.pkl', 'wb'))

# saving vectorized description
pickle.dump(tf_d, open('/content/drive/My Drive/Jobs_Recommend_Project/naukri_tf_des.pkl', 'wb'))

# saving vectorized title + description
pickle.dump(tf_td, open('/content/drive/My Drive/Jobs_Recommend_Project/naukri_tf_ti_de.pkl', 'wb'))

## stacking vectorized data

In [ ]:
from scipy.sparse import hstack

train_stack = hstack((x_tr_t, x_tr_d, x_tr_td)).tocsr()
test_stack = hstack((x_te_t, x_te_d, x_te_td)).tocsr()

## Naive Bayes

In [ ]:
%%time

# Import OneVsRestClassifier library
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

params = [{'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}]

# Creating an instance
nb = MultinomialNB()

# grid searchcv
nb_grid = GridSearchCV(estimator = nb, param_grid = params, scoring = 'f1_weighted')

# Fit the model
nb_grid.fit(train_stack, y_train)

CPU times: user 1min 55s, sys: 745 ms, total: 1min 56s
Wall time: 1min 56s


In [ ]:
print("Optimal hyperParameter:", nb_grid.best_params_)
print("Maximum accuracy:", nb_grid.best_score_ * 100, '\n')

Optimal hyperParameter: {'alpha': 0.01}
Maximum accuracy: 73.7285685900078 



## Logistic Regression

In [ ]:
%%time

# Import OneVsRestClassifier library
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV

params = [{'estimator__C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
           'estimator__penalty': ['l1', 'l2']}]

# Creating an instance
ovr_b = OneVsRestClassifier(LogisticRegression(class_weight='balanced'))

# grid searchcv
lr_grid = GridSearchCV(estimator = ovr_b, param_grid = params, scoring = 'f1_weighted')

# Fit the model
lr_grid.fit(train_stack, y_train)

In [ ]:
print("Optimal hyperParameter:", lr_grid.best_params_)
print("Maximum accuracy:", lr_grid.best_score_ * 100, '\n')

## Saving the train model in pickle file for future use.

In [ ]:
pickle.dump(lr_grid, open('/content/drive/My Drive/Jobs_Recommend_Project/naukri_logistic_reg.pkl', 'wb'))

## Predicting test data

In [ ]:
# Prediction
pred_test = lr_grid.predict(test_stack)

print("Length of predicted array:", len(pred_test), '\n\n')

print("Displaying prediction.\n")

print(pred_test)

Length of predicted array: 527 


Displaying prediction.

[13  5 13  5 26 13 13 13 13 13 13 13 13 13 13 13 13 13  5  4  7  7 13  2
  2 13 13 13 13 10 14  5  2  7 13 13 13 13 13 13 13 13  0 28 16 13 13 13
 13  3  3 13 13 13  6 13 13 13 13 13  8  0 17 13 13 13  9  9  3  3 13 13
 13  7  2 13 13  2  2 13  5 13 13 29 13  2 13 38 38 38 38 38 38 13  0 19
 13 13 13  2 17 14  2 13 13 38 17 13 13 13 13 13 38 13 13 13 13 15 13  0
 13 13 13 13 13  5 13 13 13 13  7 34 13 13 13 17 13 13 38 10 13 16 13  1
  5 13  1 13  2  8 21 13 17 13 13 13 13  8 13  6  6  2  5  5  5  5  5 11
 13 13 29  5 13 13 13 13 13 13 30 13 13 13 13 13 13 13 13 18  1  5 34 13
 16 10 33  1 13 13 13 13 13  5  5  5  5 13 13 13 13 13  9  5 28 26 26 21
 13  2 13 13 13 13  5  4 13  7 13 13 13 28 28 13 13 13 35 13 38 40 13  2
 13 13 13  5 13 33 16 13 13 13 13 13 13 22 33 13  7  2  2 21  5 10 13  5
 13 13 13 13 13 27  8 13 13 13 13 13  1 13 13 13  2 13  2 15 13 13 38 13
 13 13 13  8 13 21 13 13 10 13 13 13 13  5 13 13 13  5 13  2 13 13

### Assigning prediction encoded to test data skill's column and then shall map encoded data with the skill_val dictionary to get the corresponding skill names

In [ ]:
test_clean.head()

,jobid,jobtitle,jobdescription,skills,ti_de
224,10416500369,technical engineer el,designing chemical anchor mechanical anchors n...,NaN,technical engineer el designing chemical ancho...
304,10416600290,asst manager manager application platform secu...,job description not mentioned,NaN,asst manager manager application platform secu...
311,10416600481,hiring java developers tech leads hyderabad ze...,zenmonics global technology services product c...,NaN,hiring java developers tech leads hyderabad ze...
312,10416600490,reg urgent placement ms dynamics crm resources...,greetings we urgent req ms dynamics crm resour...,NaN,reg urgent placement ms dynamics crm resources...
320,10416600774,urgent job java j ee developer websphere appli...,we urgent positions trianz www trianz com java...,NaN,urgent job java j ee developer websphere appli...


In [ ]:
test_clean['skills'] = pred_test

test_clean.head()

,jobid,jobtitle,jobdescription,skills,ti_de
224,10416500369,technical engineer el,designing chemical anchor mechanical anchors n...,13,technical engineer el designing chemical ancho...
304,10416600290,asst manager manager application platform secu...,job description not mentioned,5,asst manager manager application platform secu...
311,10416600481,hiring java developers tech leads hyderabad ze...,zenmonics global technology services product c...,13,hiring java developers tech leads hyderabad ze...
312,10416600490,reg urgent placement ms dynamics crm resources...,greetings we urgent req ms dynamics crm resour...,5,reg urgent placement ms dynamics crm resources...
320,10416600774,urgent job java j ee developer websphere appli...,we urgent positions trianz www trianz com java...,26,urgent job java j ee developer websphere appli...


## Concat train_clean and test_clean_1
 - First, we will take only jobid and skills from the train and test data.
 - Second, we will concat them.
 - Third, we will map encoded skills to skills name
 - Fourth, we will join with the original dataframe on jobid to have skills and corresponding original jobtitle and jobdescription

In [ ]:
# First, we will take only jobid and skills from the train and test data.

train_id_skill = train_clean[['jobid', 'skills']]
test_id_skill = test_clean[['jobid', 'skills']]

# concat train_id_skill and test_id_skill
train_test = pd.concat([train_id_skill, test_id_skill])

print("Shape of train_test", train_test.shape, '\n\n')

print("Displaying head of train_test")
train_test.head()

Shape of train_test (21910, 2) 


Displaying head of train_test


,jobid,skills
0,10116001126,0
1,10116001286,1
2,10216001911,0
3,10216001918,2
4,10216005682,3


## Mapping skills encoded to get skills names

In [ ]:
train_test['skills'] = train_test['skills'].map(skills_val)

train_test.head()

,jobid,skills
0,10116001126,HR
1,10116001286,ITES
2,10216001911,HR
3,10216001918,Sales
4,10216005682,Accounts


In [ ]:
naukri_original = pd.read_csv('/content/drive/My Drive/Jobs_Recommend_Project/Copy of naukri_no_duplicate.csv')

print("Shape of naukri_original dataframe:", naukri_original.shape, '\n\n')

print("Head of naukri_original dataframe.")
naukri_original.head(2)

Shape of naukri_original dataframe: (21910, 14) 


Head of naukri_original dataframe.


,company,education,experience,industry,jobid,joblocation_address,jobtitle,numberofpositions,payrate,postdate,site_name,uniq_id,jobdescription,skills
0,Azim Premji Foundation,UG: Any Graduate - Any Specialization PG:MBA/P...,1 - 3 yrs,NGO / Social Services / Regulators / Industry ...,10116001126,Bengaluru,Recruitment Coordinator- Bangalore,NaN,Not Disclosed by Recruiter,2016-10-11 16:20:58 +0000,NaN,400dc936176f8210e9ceb99d900710e2,Job Description Send me Jobs like this Job D...,HR
1,YOGAM BPO,UG: B.Tech/B.E. - Bio-Chemistry/Bio-Technology...,0 - 1 yrs,Medical / Healthcare / Hospitals,10116001286,"Chennai Bangalore , Hyderabad Kolkata , Trivan...",Medical Coding- Fresher to CPC Coder- Whatsapp...,650.0,"2,00,000 - 3,00,000 P.A. BEST IN Industry [ Sa...",2016-09-04 18:44:08 +0000,NaN,c2673073d2442dc7ad3af2191aea899d,Job Description Send me Jobs like this YOUR ...,ITES


## joining naukri_original and train_test on job id

In [ ]:
## Removing skills column from naukri_original
naukri_original.drop('skills', axis=1, inplace=True)

In [ ]:
# joining naukri_original and train_test on jobid
naukri_final_data = naukri_original.set_index('jobid').join(train_test.set_index('jobid'))

# resetting index of naukri_final_data
naukri_final_data.reset_index(inplace=True)

print("Shape of naukri_final_data:", naukri_final_data.shape, '\n\n')

print("Displaying head of naukri_final_data.\n")
naukri_final_data.head()

Shape of naukri_final_data: (21910, 14) 


Displaying head of naukri_final_data.



,jobid,company,education,experience,industry,joblocation_address,jobtitle,numberofpositions,payrate,postdate,site_name,uniq_id,jobdescription,skills
0,10116001126,Azim Premji Foundation,UG: Any Graduate - Any Specialization PG:MBA/P...,1 - 3 yrs,NGO / Social Services / Regulators / Industry ...,Bengaluru,Recruitment Coordinator- Bangalore,NaN,Not Disclosed by Recruiter,2016-10-11 16:20:58 +0000,NaN,400dc936176f8210e9ceb99d900710e2,Job Description Send me Jobs like this Job D...,HR
1,10116001286,YOGAM BPO,UG: B.Tech/B.E. - Bio-Chemistry/Bio-Technology...,0 - 1 yrs,Medical / Healthcare / Hospitals,"Chennai Bangalore , Hyderabad Kolkata , Trivan...",Medical Coding- Fresher to CPC Coder- Whatsapp...,650.0,"2,00,000 - 3,00,000 P.A. BEST IN Industry [ Sa...",2016-09-04 18:44:08 +0000,NaN,c2673073d2442dc7ad3af2191aea899d,Job Description Send me Jobs like this YOUR ...,ITES
2,10216001911,ABC Consultants Pvt Ltd hiring for ABC Consult...,UG: Any Graduate - Any Specialization PG:Any P...,8 - 13 yrs,Construction / Engineering / Cement / Metals,"Bengaluru/Bangalore , Mumbai , Delhi/NCR(Natio...",Biz Development & Delivery- Infrastructure Pra...,2.0,"15,00,000 - 20,00,000 P.A",2016-02-19 07:23:36 +0000,NaN,52fe3ecd452d0cfd9233cb2ba255e55f,Job Description Send me Jobs like this Respo...,HR
3,10216001918,TeamLease Services Pvt Ltd,"UG: Any Graduate - Any Specialization, Graduat...",2 - 4 yrs,Travel / Hotels / Restaurants / Airlines / Rai...,Bengaluru/Bangalore,Sales Executive,NaN,"2,50,000 - 4,00,000 P.A",2016-03-30 17:03:30 +0000,NaN,b478a8b1aa1fd714de086658b07e4b99,Job Description Send me Jobs like this Good...,Sales
4,10216005682,Premium,"UG: Any Graduate - Any Specialization, Graduat...",0 - 1 yrs,Accounting / Finance,"Gurgaon , Delhi/NCR(National Capital Region) ,...","Fresher CA Job-big 4 Shared Services, Gurgaon-...",NaN,"6,00,000 - 6,50,000 P.A",2016-05-05 11:34:51 +0000,NaN,b3f20821de81b315b1f311fdc0df54ee,Job Description Send me Jobs like this Emplo...,Accounts


### Saving final dataframe for future use

In [ ]:
# naukri_final_data.to_csv('/content/drive/My Drive/Jobs_Recommend_Project/naukri_final_data_1.csv', index=False)

<center><h1>End of Naukri 1 Data Set (large)</h1>

<center><h1>Predicting skills on 2nd Naukri Data Set (small)</h1>

In [ ]:
naukri_2 = pd.read_csv('/content/drive/My Drive/Jobs_Recommend_Project/job_datasets/naukri_cleaned.csv')

print("Shape of naukri_2:", naukri_2.shape, '\n\n')

print("Displaying head of naukri_2.\n")
naukri_2.head()

Shape of naukri_2: (4120, 8) 


Displaying head of naukri_2.



,Unnamed: 0,job title,company,experience,salary,location,education,skills
0,1,Graphic Designer,"Sri Sankara Matrimony,",3 - 8 years,Not Disclosed,Bengaluru,EducationUG :B.Tech/B.E. in ComputersPG :M.Tec...,"['graphic', 'printing', 'reach', 'publishing',..."
1,2,Test Engineer Device,Reliance Jio Infocomm Limited4.0(7879 Reviews),7 - 10 years,Not Disclosed,Bengaluru,EducationUG :B.Tech/B.E. in Any Specialization...,"['ip-routing', 'wan', 'ipv6', 'lan-wan', 'prog..."
2,3,Legal Associate (2 - 7 Years) | For a Leading ...,Podhini Overseas Consultants,2 - 7 years,"₹ 3,50,000 - 6,00,000 P.A.","Chennai, Bengaluru",EducationUG :LLB in LawPG :LLM in LawDoctorate...,"['agreements', 'deeds', 'legal-research', 'non..."
3,4,ITO Svc Delivery Cons I,CSC India Pvt Ltd.3.5(587 Reviews),0 - 2 years,Not Disclosed,Bengaluru,EducationUG :Any Graduate in Any Specializatio...,"['change-management', 'sap', 'performance-mana..."
4,5,Senior Salesforce Developer with Lightning,corporate15.0(3 Reviews),2 - 5 years,Not Disclosed,Bengaluru,EducationUG :Any Graduate in Any Specializatio...,"['usage', 'pdf', 'coding', 'salesforce-dot-com..."


## Observation:
- We can see that the data set has no job description.

## Plan of action
- We will first create a new dataframe with job title, job description and skills as columns
- Job description will be filled with combination of job title, experience, education.

In [ ]:
naukri_2_df = pd.DataFrame()

naukri_2_df['jobtitle'] = naukri_2['job title']
naukri_2_df['jobdescription'] = naukri_2['job title'] + ' ' + naukri_2['experience'] + ' ' + naukri_2['education']
naukri_2_df['skills'] = ''

naukri_2_df.head()

,jobtitle,jobdescription,skills
0,Graphic Designer,Graphic Designer 3 - 8 years EducationUG :B.Te...,
1,Test Engineer Device,Test Engineer Device 7 - 10 years EducationUG ...,
2,Legal Associate (2 - 7 Years) | For a Leading ...,Legal Associate (2 - 7 Years) | For a Leading ...,
3,ITO Svc Delivery Cons I,ITO Svc Delivery Cons I 0 - 2 years EducationU...,
4,Senior Salesforce Developer with Lightning,Senior Salesforce Developer with Lightning 2 -...,


In [ ]:
naukri_2_dfc = copy.deepcopy(naukri_2_df)

## Data cleaning

In [ ]:
def clean_2(text):
    te = re.sub('[^a-zA-Z]', ' ', text)
    # te = ' '.join(lem.lemmatize(sw) for sw in te.split() if sw not in stopwords.words('english'))
    te = ' '.join(sw for sw in te.split() if sw not in stopwords.words('english'))
    te = te.lower().strip()

    return te

In [ ]:
from tqdm import tqdm
clean_title_2 = []

for i, s in enumerate (tqdm(naukri_2_dfc['jobtitle'])):
    t = clean_2(s)
    clean_title_2.append(t)

100%|██████████| 4120/4120 [00:02<00:00, 1923.80it/s]


In [ ]:
from tqdm import tqdm
clean_des_2 = []

for i, s in enumerate (tqdm(naukri_2_dfc['jobdescription'])):
    t = clean_2(s)
    clean_des_2.append(t)

100%|██████████| 4120/4120 [00:08<00:00, 464.63it/s]


In [ ]:
print("Assigning clean text to title and description columns...\n")

naukri_2_dfc['jobtitle']= clean_title_2
naukri_2_dfc['jobdescription'] = clean_des_2

naukri_2_dfc.head()

Assigning clean text to title and description columns...



,jobtitle,jobdescription,skills
0,graphic designer,graphic designer years educationug b tech b e ...,
1,test engineer device,test engineer device years educationug b tech ...,
2,legal associate years for leading law firm che...,legal associate years for leading law firm che...,
3,ito svc delivery cons i,ito svc delivery cons i years educationug any ...,
4,senior salesforce developer lightning,senior salesforce developer lightning years ed...,


## Featuring engineering.
- Adding title and description

In [ ]:
# feature engineering adding title and description
naukri_2_dfc['ti_de'] = naukri_2_dfc['jobtitle'] + ' ' + naukri_2_dfc['jobdescription']

naukri_2_dfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4120 entries, 0 to 4119
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   jobtitle        4120 non-null   object
 1   jobdescription  4120 non-null   object
 2   skills          4120 non-null   object
 3   ti_de           4120 non-null   object
dtypes: object(4)
memory usage: 128.9+ KB


## Vectorizing title, description and title+description columns

In [ ]:
# transforming title column
tf_t_2 = tf_t.transform(naukri_2_dfc['jobtitle'])

# transforming description column
tf_d_2 = tf_d.transform(naukri_2_dfc['jobdescription'])

# transforming title+description column
tf_td_2 = tf_td.transform(naukri_2_dfc['ti_de'])

## Stacking vectorized data

In [ ]:
stack_naukri_2 = hstack((tf_t_2, tf_d_2, tf_td_2)).tocsr()

## Prediction

In [ ]:
pred_naukri_2 = lr_grid.predict(stack_naukri_2)

print("Length of predicted array:", len(pred_naukri_2), '\n\n')

print("Displaying 20 prediction.\n")

print(pred_naukri_2[:20])

Length of predicted array: 4120 


Displaying 20 prediction.

[ 6 11 30  5  5 30 11  5 13  8 38  5 38  5  1  5 38 12  5 34]


## Assiging prediction to the skills column in naukri_2_df

In [ ]:
naukri_2_df['skills'] = pred_naukri_2

## Mapping skills encoded to get skill names

In [ ]:
naukri_2_df['skills'] = naukri_2_df['skills'].map(skills_val)

## Assigning few columns or original dataframe (naukri_2) to new dataframe (naukri_2_df)

In [ ]:
print("Columns in naukri_2")
print(naukri_2.columns, '\n')

print("Columns in naukri_2_df")
print(naukri_2_df.columns, '\n')

Columns in naukri_2
Index(['Unnamed: 0', 'job title', 'company', 'experience', 'salary',
       'location', 'education', 'skills'],
      dtype='object') 

Columns in naukri_2_df
Index(['jobtitle', 'jobdescription', 'skills', 'company', 'experience',
       'salary', 'location', 'education'],
      dtype='object') 



In [ ]:
naukri_2_df['company'] = naukri_2['company']
naukri_2_df['experience'] = naukri_2['experience']
naukri_2_df['salary'] = naukri_2['salary']
naukri_2_df['location'] = naukri_2['location']
naukri_2_df['education'] = naukri_2['education']

In [ ]:
naukri_2_df = naukri_2_df[['jobtitle', 'jobdescription', 'company', 'experience', 'salary', 'location', 'education', 'skills']]

In [ ]:
print("Shape of naukri_2_df:", naukri_2_df.shape, '\n\n')

print("Displaying head of naukri_2_df.\n")
naukri_2_df.head()

Shape of naukri_2_df: (4120, 8) 


Displaying head of naukri_2_df.



,jobtitle,jobdescription,company,experience,salary,location,education,skills
0,Graphic Designer,Graphic Designer 3 - 8 years EducationUG :B.Te...,"Sri Sankara Matrimony,",3 - 8 years,Not Disclosed,Bengaluru,EducationUG :B.Tech/B.E. in ComputersPG :M.Tec...,Design
1,Test Engineer Device,Test Engineer Device 7 - 10 years EducationUG ...,Reliance Jio Infocomm Limited4.0(7879 Reviews),7 - 10 years,Not Disclosed,Bengaluru,EducationUG :B.Tech/B.E. in Any Specialization...,IT Software - QA & Testing
2,Legal Associate (2 - 7 Years) | For a Leading ...,Legal Associate (2 - 7 Years) | For a Leading ...,Podhini Overseas Consultants,2 - 7 years,"₹ 3,50,000 - 6,00,000 P.A.","Chennai, Bengaluru",EducationUG :LLB in LawPG :LLM in LawDoctorate...,Legal
3,ITO Svc Delivery Cons I,ITO Svc Delivery Cons I 0 - 2 years EducationU...,CSC India Pvt Ltd.3.5(587 Reviews),0 - 2 years,Not Disclosed,Bengaluru,EducationUG :Any Graduate in Any Specializatio...,IT Software - Application Programming
4,Senior Salesforce Developer with Lightning,Senior Salesforce Developer with Lightning 2 -...,corporate15.0(3 Reviews),2 - 5 years,Not Disclosed,Bengaluru,EducationUG :Any Graduate in Any Specializatio...,IT Software - Application Programming


In [ ]:
# saving the final data frame
# naukri_2_df.to_csv('/content/drive/My Drive/Jobs_Recommend_Project/naukri_final_data_2.csv', index=False)

<center><h1>Job Recommendation based on keywords</h1>

In [ ]:
naukri_dup = pd.read_csv('/content/drive/My Drive/Jobs_Recommend_Project/naukri_final_data_1.csv')

In [ ]:
print("Shape of naukri_dup:", naukri_dup.shape)

print("\nDisplaying head of the dataframe\n")
naukri_dup.head()

Shape of naukri_dup: (21910, 14)

Displaying head of the dataframe



,jobid,company,education,experience,industry,joblocation_address,jobtitle,numberofpositions,payrate,postdate,site_name,uniq_id,jobdescription,skills
0,10116001126,Azim Premji Foundation,UG: Any Graduate - Any Specialization PG:MBA/P...,1 - 3 yrs,NGO / Social Services / Regulators / Industry ...,Bengaluru,Recruitment Coordinator- Bangalore,NaN,Not Disclosed by Recruiter,2016-10-11 16:20:58 +0000,NaN,400dc936176f8210e9ceb99d900710e2,Job Description Send me Jobs like this Job D...,HR
1,10116001286,YOGAM BPO,UG: B.Tech/B.E. - Bio-Chemistry/Bio-Technology...,0 - 1 yrs,Medical / Healthcare / Hospitals,"Chennai Bangalore , Hyderabad Kolkata , Trivan...",Medical Coding- Fresher to CPC Coder- Whatsapp...,650.0,"2,00,000 - 3,00,000 P.A. BEST IN Industry [ Sa...",2016-09-04 18:44:08 +0000,NaN,c2673073d2442dc7ad3af2191aea899d,Job Description Send me Jobs like this YOUR ...,ITES
2,10216001911,ABC Consultants Pvt Ltd hiring for ABC Consult...,UG: Any Graduate - Any Specialization PG:Any P...,8 - 13 yrs,Construction / Engineering / Cement / Metals,"Bengaluru/Bangalore , Mumbai , Delhi/NCR(Natio...",Biz Development & Delivery- Infrastructure Pra...,2.0,"15,00,000 - 20,00,000 P.A",2016-02-19 07:23:36 +0000,NaN,52fe3ecd452d0cfd9233cb2ba255e55f,Job Description Send me Jobs like this Respo...,HR
3,10216001918,TeamLease Services Pvt Ltd,"UG: Any Graduate - Any Specialization, Graduat...",2 - 4 yrs,Travel / Hotels / Restaurants / Airlines / Rai...,Bengaluru/Bangalore,Sales Executive,NaN,"2,50,000 - 4,00,000 P.A",2016-03-30 17:03:30 +0000,NaN,b478a8b1aa1fd714de086658b07e4b99,Job Description Send me Jobs like this Good...,Sales
4,10216005682,Premium,"UG: Any Graduate - Any Specialization, Graduat...",0 - 1 yrs,Accounting / Finance,"Gurgaon , Delhi/NCR(National Capital Region) ,...","Fresher CA Job-big 4 Shared Services, Gurgaon-...",NaN,"6,00,000 - 6,50,000 P.A",2016-05-05 11:34:51 +0000,NaN,b3f20821de81b315b1f311fdc0df54ee,Job Description Send me Jobs like this Emplo...,Accounts


In [ ]:
naukri_dup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21910 entries, 0 to 21909
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   jobid                21910 non-null  int64  
 1   company              21906 non-null  object 
 2   education            19917 non-null  object 
 3   experience           21906 non-null  object 
 4   industry             21905 non-null  object 
 5   joblocation_address  21420 non-null  object 
 6   jobtitle             21910 non-null  object 
 7   numberofpositions    4445 non-null   float64
 8   payrate              21813 non-null  object 
 9   postdate             21887 non-null  object 
 10  site_name            3967 non-null   object 
 11  uniq_id              21910 non-null  object 
 12  jobdescription       21910 non-null  object 
 13  skills               21910 non-null  object 
dtypes: float64(1), int64(1), object(12)
memory usage: 2.3+ MB


## vectorization of title

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_td = TfidfVectorizer(ngram_range=(1,2))

tf_title_des_fit = tf_td.fit_transform(naukri_dup['jobtitle'] + ' ' + naukri_dup['jobdescription'])

In [ ]:
naukri_dup.reset_index(inplace=True, drop=True)

## Function to get pariwise distance and recommend jobs

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

def job_recommend(input_title, tf_ti_de, vec_title_des, dataframe, number_recommend):

    title = [input_title]
    vec_input = tf_ti_de.transform(title)

    pairwise_dist = pairwise_distances(vec_title_des, vec_input)
    indices = np.argsort(pairwise_dist.flatten())
    df_indices = list(dataframe.index[indices])

    print("\nTop {} job recommendation for the job title {}".format(number_recommend, input_title))
    print('-'*80, '\n')

    for i in range(number_recommend):
        print('Company:', naukri_dup['company'].loc[df_indices[i]])
        print('Job_id:', naukri_dup['jobid'].loc[df_indices[i]])
        print('Title:', naukri_dup['jobtitle'].loc[df_indices[i]])
        print('Description:', naukri_dup['jobdescription'].loc[df_indices[i]])
        print('Skills:', naukri_dup['skills'].loc[df_indices[i]])
        print('Job location:', naukri_dup['joblocation_address'].loc[df_indices[i]], '\n')
        print('-'*50, '\n')


In [ ]:
job_recommend('data science', tf_td, tf_title_des_fit, naukri_dup, 5)


Top 5 job recommendation for the job title data science
-------------------------------------------------------------------------------- 

Company: Rinalytics Advisors Pvt. Ltd
Job_id: 250515500955
Title: Data Science Role
Description: Job Description   Send me Jobs like this Job Description
Skills: Analytics & Business Intelligence
Job location: Bengaluru/Bangalore 

-------------------------------------------------- 

Company: GE India Technology Pvt. Ltd.
Job_id: 300316900542
Title: Data Analytics Manager
Description: Job Description   Send me Jobs like this Role Summary/Purpose: The Data Analytics Manager will be responsible for program execution with broad analytics scope and high impact to the GE Global Operations business. In this role, you will be responsible for leading, executing and operationalization of world class data driven methods, advanced algorithms and capabilities to support the needs of GE businesses. You will be responsible for identifying new opportunities and p

In [ ]:
job_recommend('sales manager', tf_td, tf_title_des_fit, naukri_dup, 5)


Top 5 job recommendation for the job title sales manager
-------------------------------------------------------------------------------- 

Company: HITECH PLACEMENTS
Job_id: 250515500094
Title: SALES MANAGER
Description: Job Description   Send me Jobs like this Job Description:
Skills: Sales
Job location: Bengaluru/Bangalore 

-------------------------------------------------- 

Company: Sanskriti Consultancy hiring for Sales Manager/ Sr Sales Manager/ Agency Sales Manager- Life Insurance- Pan India
Job_id: 241216000808
Title: Direct Sales Manager/ Agency Sales Manager- Life Insurance- Pan India
Description: Job Description   Send me Jobs like this We have some very good Openings in one of the top Life Insurance Companies. Designation: Sales Manager / Senior Sales Manager / Asst. Sales Manager- Agency/ Direct/ Banca Channel-Life Insurance. Locations: Pan India. Salary: Negotiable + Incentives. Salary:INR 2,00,000 - 4,00,000 P.A. Salary Negotiable+ Incentives Industry: Insurance Funct

In [ ]:
job_recommend('quality testing', tf_td, tf_title_des_fit, naukri_dup, 5)


Top 5 job recommendation for the job title quality testing
-------------------------------------------------------------------------------- 

Company: OKDA Solutions
Job_id: 211216900526
Title: Automation Test Engineer - Software Testing Life Cycle
Description: Job Description   Send me Jobs like this Immediate joiners or 15 Days notice period candidates only . Skills : - Knowledge of Software Testing Life Cycle. Work experience in all phases of testing life cycle i.e. Test Planning, Test Case Design, Test Execution and Defect Management. - Good experience in Functional Testing, Integration Testing, Security Testing, Regression Testing, Browser Compatibility Testing and User Acceptance testing. - Skills in Automation Testing, MS VSTS /Visual Studio, Coded UI, CUITe - Knowledge of other relevant tools for DotNet web applications testing - C#, SQL, Agile Methodology, SOAP UI, Knowledge of MTM, TFS. Salary: Not Disclosed by Recruiter Industry: IT-Software / Software Services Functional A

In [ ]:
job_recommend('verification', tf_td, tf_title_des_fit, naukri_dup, 5)


Top 5 job recommendation for the job title verification
-------------------------------------------------------------------------------- 

Company: ONS Search
Job_id: 280515500993
Title: SMTS / Verification Manager IP Verification
Description: Job Description   Send me Jobs like this Responsibilities: Besides people responsibility the design verification manager/SMTS is expected to have strong functional verification domain expertise         to provide guidance to the team. Responsible for verification at IP level Create testplans for verification of the various features and testplan execution Develop assembly level directed tests as needed for the feature Develop coverpoints and perform coverage analysis Develop verification components using OVM/UVM and System Verilog Actively participate in the architectural discussions and provide feedback to RTL as needed Being a mentor and technical leader for more junior verification engineers. Leading and participating in IP level verification 

In [ ]:
job_recommend('java', tf_td, tf_title_des_fit, naukri_dup, 5)


Top 5 job recommendation for the job title java
-------------------------------------------------------------------------------- 

Company: Xecutesmart
Job_id: 220915003163
Title: Online Trainer - Java,j2ee,servlets,ejb,jsp,struts - Part-time
Description: Job Description   Send me Jobs like this Hi, We are looking for online trainer, who can work as a part-time. Java,J2EE Technologies: Core Java Advance Java Servlets Java Beans JSP J2EE : Struts Spring EJB XML JMS Hibernate Web Services Java Mail AJAX Maverick SOA Ruby on Rails Salary:INR 1,00,000 - 5,00,000 P.A Industry: IT-Software / Software Services Functional Area: IT Software - Application Programming , Maintenance Role Category:Programming & Design Role:Software Developer Keyskills Java J2EE jsp Java Programming J2Ee Spring Java Beans Java Swing Java Servlets Java Struts Java Server Pages Java Applets Java Awt Java Script Java Scripting Hibernate JMS XML Ruby on Rails AJAX Desired Candidate Profile Education- UG: Any Graduate -

In [ ]:
job_recommend('php', tf_td, tf_title_des_fit, naukri_dup, 5)


Top 5 job recommendation for the job title php
-------------------------------------------------------------------------------- 

Company: Aaxeleron Technology Solutions Pvt. Ltd.
Job_id: 281116003257
Title: Immediate Openings for PHP Developer in BLR with our Reputed Client
Description: Job Description   Send me Jobs like this Hands on experience into PHP development. Interview Location : Bangalore Job Location : Bangalore Mode of hire : Permanent Regards roja Salary: Not Disclosed by Recruiter Industry: IT-Software / Software Services Functional Area: IT Software - Application Programming , Maintenance Role Category:Programming & Design Role:Software Developer Keyskills PHP developer PHP development php programmer php consultant php Desired Candidate Profile Please refer to the Job description above Education- UG: Any Graduate - Any Specialization Company Profile: Aaxeleron Technology Solutions Pvt. Ltd. Leading IT recruitment Firm Download PPT Photo 1   View Contact Details
Skills:

In [ ]:
job_recommend('sql', tf_td, tf_title_des_fit, naukri_dup, 5)


Top 5 job recommendation for the job title sql
-------------------------------------------------------------------------------- 

Company: Ikya Human Capital Solutions Pvt Ltd hiring for HCL Technologies Limited
Job_id: 291115000015
Title: Opportunity for SQL DBA - 5th Nov 2015 (saturday) - Noida
Description: Job Description   Send me Jobs like this Greeting from IKYA..!!! This is Nirmalraj from IKYA global.  I have an excellent opportunity with at Top Infra company for SQL DBA at Chennai & Noida Kindly have a look in the below interview details, Permanent payroll with HCL Interview Date: 5th December 2015 (Saturday) Interview Location: Noida Joining location: Chennai & Noida Experience: 4 - 12 years L2: Relevant Experience - 4 years L3: Relevant Experience - 6 years Multiple openings going on - Interested candidates apply ASAP Only 2 rounds of Interview - Online & technical interview + HR discussion Interested candidates can send me your updated profile along with the following detai

In [ ]:
job_recommend('mechanical', tf_td, tf_title_des_fit, naukri_dup, 5)


Top 5 job recommendation for the job title mechanical
-------------------------------------------------------------------------------- 

Company: Phillips HVC consulting Pvt Ltd
Job_id: 210715002557
Title: Mechanical Engineer / Diploma- Freshers Also Apply
Description: Job Description   Send me Jobs like this Lead a team of Mechanical Engineers. Has worked as Responsible Engineer, Lead Engineer and Reviewer. Educational and Other Qualification-Degree or Diploma IN Mechanical Engineering. Salary:INR 3,00,000 - 7,00,000 P.A. Include TA+ D.A +HRA Industry: Automobile / Auto Anciliary / Auto Components Functional Area: Site Engineering , Project Management Role Category:Site Engineering Role:Mechanical Engineer-Telecom Keyskills Mechanical Engineering mechanical construction mechanical manufacturing be btech diploma iti degree b.e mechanical b.tech +2 tele female Mechanical Engineer diploma mechanical mechanical design engineer machine design engineer Desired Candidate Profile Education- 

In [ ]:
job_recommend('electrical', tf_td, tf_title_des_fit, naukri_dup, 5)


Top 5 job recommendation for the job title electrical
-------------------------------------------------------------------------------- 

Company: XCEL / OQTS
Job_id: 300915005560
Title: Electrical Engineer - Facility Management
Description: Job Description   Send me Jobs like this Manage & supervise the electrical requirements of the Project Manage optimum inventory of Electrical materials Identify & Solve electrical problems Rectify problems & complaints Work as per the health & safety regulations Maintain exact reporting Salary: Not Disclosed by Recruiter Industry: Construction / Engineering / Cement / Metals Functional Area: Site Engineering , Project Management Role Category:Site Engineering Role:Electrical Engineer-Utility Keyskills Electrical Engineering BE Electrical Diploma in Electrical Electrical contracting Construction electricians ELectrical supervisors for construction Electricians Site Management- electrical b license c license C Licence B Licence Desired Candidate Prof

In [ ]:
job_recommend('civil', tf_td, tf_title_des_fit, naukri_dup, 5)


Top 5 job recommendation for the job title civil
-------------------------------------------------------------------------------- 

Company: Al Raheema Agencies hiring for A Leading company in Marine Industry
Job_id: 60516000964
Title: Site Engineer
Description: Job Description   Send me Jobs like this Civil background with 5-8 years experience in Marine related work, capable of studying Drawing etc. Salary:INR Best in the Industry and negotiable according to the candidate. Industry: Other Functional Area: Site Engineering , Project Management Role Category:Site Engineering Role:Civil Engineer-Land Development Keyskills Construction Management Civil Civil Engineering Construction Planning Construction Supervision Marine marine civil manager Civil Structural Designing Project - Civil & Engineering civil site engineer Engineering Drawing Desired Candidate Profile Education- UG: B.Tech/B.E. - Civil, Diploma - Civil PG:Any Postgraduate - Any Specialization, Post Graduation Not Required Do

In [ ]:
job_recommend('sap', tf_td, tf_title_des_fit, naukri_dup, 5)


Top 5 job recommendation for the job title sap
-------------------------------------------------------------------------------- 

Company: HR Indeed
Job_id: 50816900850
Title: SAP Consultant- German Speaking
Description: Job Description   Send me Jobs like this Required minimum 4 Years of SAP experience in below modules, consultants with German Speaking: Skill SAP ABAP SAP FI SAP CO SAP BW SAP GRC SAP PP SAP MM SAP SD SAP CRM SAP Basis DC Salary: Not Disclosed by Recruiter Industry: IT-Software / Software Services Functional Area: IT Software - ERP , CRM Role Category:System Design/Implementation/ERP/CRM Role:Functional Outside Consultant Keyskills SAP SD SAP MM SAP PP SAP FI SAP ABAP SAP Consulting SAP GRC SAP CRM SAP Basis SAP CO SAP German Desired Candidate Profile Education- UG: B.Tech/B.E. - Any Specialization, Computers PG:Any Postgraduate - Any Specialization, Post Graduation Not Required Doctorate:Doctorate Not Required   Company Profile: HR Indeed A Leading client of HR Indee

In [ ]:
job_recommend('IT Software - DBA', tf_td, tf_title_des_fit, naukri_dup, 5)


Top 5 job recommendation for the job title IT Software - DBA
-------------------------------------------------------------------------------- 

Company: Confidential
Job_id: 90516900388
Title: Oracle DBA
Description: Job Description   Send me Jobs like this Required skills: Oracle DBA Experience Required: 5 To 10years Notice period: 10-15 days Job Location: Chennai, Salary: Not Disclosed by Recruiter Industry: Recruitment / Staffing Functional Area: IT Software - DBA , Datawarehousing Role Category:Admin/Maintenance/Security/Datawarehousing Role:DBA Keyskills Database Administration Oracle DBA DBA Desired Candidate Profile   Education- UG: Any Graduate - Any Specialization Company Profile: Confidential Confidential Download PPT Photo 1   View Contact Details
Skills: IT Software - DBA
Job location: Bengaluru 

-------------------------------------------------- 

Company: Unitforce technologies Pvt. Ltd.
Job_id: 160316600408
Title: Oracle DBA
Description: Job Description   Send me Jobs 